In [1]:
pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, log_loss
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Prüfe ob eine CPU für das Training verfügbar ist.

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Um die Ergebnisse reproduzierbar zu machen erstellen wir einen random_state und intialisieren die seeds für die RNG.

In [4]:
random_state = 1
tf.random.set_seed(random_state)
np.random.seed(random_state)

Lade die erstellten Trainingsdaten.

In [9]:
data = pd.read_csv('/content/drive/MyDrive/WR2 Brrr/Trainingsdaten_Proj3/training_dataset_ver6.csv', header = None)
print(data.shape)
data.head()

(42160, 4801)


,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,1,1,1,1,1,1,1,1,...,0,0,1,1,1,1,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,1,1,1,...,1,1,0,0,0,0,0,1,1,1


Unterteile die Trainingsdaten in den Input für das Neuronale Netz und die zugehörigen label:

In [10]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1:].values

Lege die Parameterverteilungen für den GridSearch fest und erstelle das zugehörige dictionary. In der Variable metrics werden die gewünschten Metriken eingetragen, welche während des fittings evaluiert werden. Da diese aber nicht direkt im GridSearch eingesehen werden können, werden sie vorerst nicht benötigt.

In [11]:
batch_size      = [16,32]
epochs          = 300
optimizer       = 'sgd'
learning_rate   = [0.001,0.01]
n_hidden_layers = [5]#[3,5,6]
layer_size      = [1600,3200]
dropout_rate    = [0.2, 0.3]

parameters = {
    'fit__batch_size': batch_size,
    'optimizer__learning_rate' : learning_rate,
    'model__n_hidden_layers' : n_hidden_layers,
    'model__layer_size' : layer_size,
    'model__dropout_rate' : dropout_rate
    }

Der KerasClassifier Wrapper nimmt eine build_fn als Argument, in welcher das Keras Modell erstellt wird. Erstelle in dieser die Netzwerk Architektur und lege die möglichen Hyperparameter fest.

In unserem Fall ist die Modell Architektur ein dichtes FFN welches folgendermaßen aufgebaut ist:
Input -> DenseLayer -> ActivationFN -> DropoutLayer -> DenseLayer -> ... -> DropoutLayer -> OutputLayer(1 Neuron) -> SigmoidFN

In [12]:
def build_model(n_hidden_layers, layer_size, dropout_rate):
    model = tf.keras.models.Sequential()
    model.add(
        Dense(
          units=layer_size,
          activation = 'relu',
          input_shape=(4800,),
          name = 'dense_1'
        )
    )

    for i in range(n_hidden_layers-1):
        model.add(tf.keras.layers.Dropout(dropout_rate,seed = random_state+i, name = 'dropout_' + str(i+1))) 
        model.add(Dense(units=layer_size, activation = 'relu', name = 'dense_' + str(i+2)))
        

    # Output Layer
    model.add(Dense(1, activation = 'sigmoid', name = 'output'))
        
    return model

cb = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                       min_delta=0.0001, 
                                       patience=20, 
                                       verbose=1, 
                                       mode='auto',  
                                       restore_best_weights=True)]

model_kwargs = dict(model=build_model,
                    loss="binary_crossentropy",
                    optimizer = optimizer,
                    optimizer__learning_rate = 0.001,
                    model__n_hidden_layers = 5,
                    model__layer_size = 500,
                    model__dropout_rate = 0.2,
                    random_state = random_state,
                    verbose = 0,
                    fit__validation_split=0.25,
                    fit__epochs = epochs,
                    fit__batch_size = 64,
                    callbacks = cb
)
    
model = KerasClassifier(**model_kwargs)

Cross-Validation Strategy. Da die generierten Trainingsdaten recht umfangreich sind, eignet sich ein 4er-Split. Der CV wird zudem mit einem random state versehen.

In [13]:
cv = StratifiedKFold(n_splits = 4, shuffle = True, random_state = random_state)

Das Format in welchem der Output in tf.Keras generiert werden macht es notwendig, dass der scikit-learn Cross-Entropy scorer mit einer Toleranz versehen wird.

In [14]:
float32log_loss = make_scorer(log_loss, eps = 1e-7)

## GridSearch


---


Erstelle den GridSearchCV. Interessante Auswertungen sind die Genauigkeit, der Cross-Entropy loss sowie der Recall. Erstelle zudem einen zusätzlichen random state für die Suche, um mehrere kürzere Durchlaufe starten zu können, ohne dabei andere Parameter zu beeinflussen.

In [15]:
gridsearch = GridSearchCV(estimator = model,
                          param_grid = parameters,
                          scoring={'accuracy':'accuracy',
                                  'ce':float32log_loss,
                                  'recall': 'recall'},   
                          refit = False,
                          cv=cv,
                          return_train_score = False,
                          verbose = 4)

Führe die GridSearch durch und speichere die Ergebnisse ab.

In [ ]:
gridsearch.fit(X, y, verbose = 0,validation_split = 0.25)
result_df = pd.DataFrame.from_dict(gridsearch.cv_results_)
result_df.to_pickle('/content/drive/MyDrive/WR2 Brrr/Trainingsdaten_Proj3/gridsearch/gridsearch_results_nhl'+ str(n_hidden_layers[0])+ '.pkl')
result_df.head()

Fitting 4 folds for each of 16 candidates, totalling 64 fits


### Ergebnisse aus verschiedenen Durchläufen zusammenführen

In [6]:
rs = [0,1,2,3,10,11,12]
names = ['randsearch_results_rs'+ str(rs_)+ '.pkl' for rs_ in rs]
dfs = []
for name in names:
  dfs.append(pd.read_pickle('/content/drive/MyDrive/WR2 Brrr/Trainingsdaten_Proj3/randomsearch/' + name))

results = pd.concat(dfs, axis = 'index', ignore_index = True).sort_values(by = ["mean_test_accuracy"])
results

FileNotFoundError: ignored

Speichern der Ergebnisse

In [ ]:
results.to_csv('/content/drive/MyDrive/WR2 Brrr/Trainingsdaten_Proj3/randomsearch_results.csv')